<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Language Models

**Note: For other Non-OpenAI models, you can check out: https://python.langchain.com/docs/modules/model_io/models/llms/ although the interface is extremely similar, its just that the results from .generation calls will have differentinformation depending on the service you use.**

## Text Model Connection

In [20]:
from langchain.llms import OpenAI
from langchain_openai import AzureOpenAI
import os

You can stored your API key however you prefer, its common to set it as an environment variable, for example:

In [25]:



os.environ['OpenAI_API_KEY'] = 'ce53c5fce80c4503927244333e40634c'
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wdc-chat-llm.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = "ce53c5fce80c4503927244333e40634c"
os.environ["OPENAI_API_VERSION"] = "2024-02-01"
SecretStr = os.environ["AZURE_OPENAI_API_KEY"]

In [26]:
print(os.environ['OPenAI_API_KEY'])
print(os.environ["AZURE_OPENAI_API_KEY"])

ce53c5fce80c4503927244333e40634c
ce53c5fce80c4503927244333e40634c


Note, that LangChain automatically looks up for any environment variable with the name OPENAI_API_KEY automatically when making a connection to OpenAI. Alternatively, you could just pass in the openai key via a string (not very secure, but okay for your own local projects), or even just save it somewhere on your computer in a text file and then read it in, for example:

In [5]:
f = open('C:\\Users\\Marcial\\Desktop\\desktop_openai.txt')
api_key = f.read()

In [27]:
#llm = OpenAI(openai_api_key=api_key)
llm = AzureOpenAI(model="gpt -3.5-turbo-instruct")

## Text Model Call

This is the simplest way to get a text autocomplete:

In [28]:
print(llm('Here is a fun fact about Pluto:'))

NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}

You can also use generate to get full output with more info:

In [8]:
# NEEDS TO BE A LIST, EVEN FOR JUST ONE STRING
result = llm.generate(['Here is a fun fact about Pluto:',
                     'Here is a fun fact about Mars:']
                     )

In [9]:
result.schema()

{'title': 'LLMResult',
 'description': 'Class that contains all results for a batched LLM call.',
 'type': 'object',
 'properties': {'generations': {'title': 'Generations',
   'type': 'array',
   'items': {'type': 'array', 'items': {'$ref': '#/definitions/Generation'}}},
  'llm_output': {'title': 'Llm Output', 'type': 'object'},
  'run': {'title': 'Run',
   'type': 'array',
   'items': {'$ref': '#/definitions/RunInfo'}}},
 'required': ['generations'],
 'definitions': {'Generation': {'title': 'Generation',
   'description': 'A single text generation output.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'generation_info': {'title': 'Generation Info', 'type': 'object'}},
   'required': ['text']},
  'RunInfo': {'title': 'RunInfo',
   'description': 'Class that contains metadata for a single execution of a Chain or model.',
   'type': 'object',
   'properties': {'run_id': {'title': 'Run Id',
     'type': 'string',
     'format': 'uuid'}},
   'requ

In [10]:
result.llm_output

{'token_usage': {'completion_tokens': 49,
  'total_tokens': 65,
  'prompt_tokens': 16},
 'model_name': 'text-davinci-003'}

# Chat Models

The most popular models are actually chat models, that have a System Message and then a series of Assistant and Human Messages

In [11]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(openai_api_key=api_key)

In [12]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
result = chat([HumanMessage(content="Can you tell me a fact about Earth?")])

In [14]:
result

AIMessage(content='One fact about Earth is that it is the only known planet to support life.', additional_kwargs={}, example=False)

In [15]:
result.content

'One fact about Earth is that it is the only known planet to support life.'

In [16]:
result = chat([SystemMessage(content='You are a very rude teenager who only wants to party and not answer questions'),
               HumanMessage(content='Can you tell me a fact about Earth?')])

In [17]:
result.content

"Ugh, seriously? Fine, here's a fact for you. Earth is the third planet from the Sun in our solar system. Now, can we move on to something more exciting? Like, I don't know, partying or something?"

In [18]:
# NEEDS TO BE A LIST!
result = chat.generate(
                [
                [SystemMessage(content='You are a University Professor'),
               HumanMessage(content='Can you tell me a fact about Earth?')]
                ]
                    )

In [19]:
result

LLMResult(generations=[[ChatGeneration(text='Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.', generation_info=None, message=AIMessage(content='Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 25, 'completion_tokens': 62, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('075174db-b350-4076-8b34-f43ea59169dd'))])

In [20]:
result.llm_output

{'token_usage': {'prompt_tokens': 25,
  'completion_tokens': 62,
  'total_tokens': 87},
 'model_name': 'gpt-3.5-turbo'}

In [21]:
result.generations[0][0].text

'Certainly! One interesting fact about Earth is that it is the only known planet in our solar system where water exists in all three states: solid (ice), liquid (water), and gas (water vapor). This unique combination of water in different states allows for the existence of life as we know it on Earth.'

## Extra Parameters and Args

Here we add in some extra parameters and args, note we chose some pretty extreme values!

In [22]:
result = chat([HumanMessage(content='Can you tell me a fact about Earth?')],
                 temperature=2,presence_penalty=1,max_tokens=100)

In [25]:
result.content

'Cycling remains one of the mosPopular physical activities performed on Earth'

# Caching

Making the same exact request often? You could use a cache to store results **note, you should only do this if the prompt is the exact same and the historical replies are okay to return**.

In [26]:
import langchain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(openai_api_key=api_key)

In [27]:
from langchain.cache import InMemoryCache
langchain.llm_cache = InMemoryCache()

# The first time, it is not yet in cache, so it should take longer
llm.predict("Tell me a fact about Mars")

'One interesting fact about Mars is that it has the largest volcano in the solar system. Named Olympus Mons, this shield volcano stands about 13.6 miles (22 kilometers) high and spans approximately 370 miles (600 kilometers) in diameter. It is nearly three times the height of Mount Everest, making it the tallest volcano and one of the largest known volcanoes in the entire solar system.'

In [28]:
# You will notice this reply is instant!
llm.predict('Tell me a fact about Mars')

'One interesting fact about Mars is that it has the largest volcano in the solar system. Named Olympus Mons, this shield volcano stands about 13.6 miles (22 kilometers) high and spans approximately 370 miles (600 kilometers) in diameter. It is nearly three times the height of Mount Everest, making it the tallest volcano and one of the largest known volcanoes in the entire solar system.'

You can also use SQLite Caches: https://python.langchain.com/docs/modules/model_io/models/chat/how_to/chat_model_caching#sqlite-cache